In [ ]:
import os
import random
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import torch

In [ ]:
IMAGES_PATH = Path("imgs/finetune_gpt/")
DATA_PATH = Path("data/finetune_gpt/")

IMAGES_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH.mkdir(parents=True, exist_ok=True)


def seed_all(seed: int) -> None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)


SEED = 42

seed_all(SEED)

In [ ]:
from datasets import load_dataset

# Загрузка датасета
dataset = load_dataset(
    "d0rj/geo-reviews-dataset-2023", cache_dir=DATA_PATH / "model_cache"
)

In [ ]:
# Преобразование данных в DataFrame
data_df = pd.DataFrame(dataset["train"])

print("Number of rows and columns in the data set:", data_df.shape)

In [ ]:
data_df.head(5)

In [ ]:
data_df.info()

In [ ]:
work_data = data_df.dropna(subset=["text", "name_ru", "rating"])
work_data = work_data.drop_duplicates(subset=["text"]).reset_index(drop=True)
work_data["text"] = work_data["text"].str.replace("\\n", " ")
work_data = work_data[:50000]
work_data["text"][0]

In [ ]:
work_data.info()

In [ ]:
unique_name_ru = work_data["name_ru"].unique().tolist()

unique_rubrics = work_data["rubrics"].unique().tolist()

In [ ]:
from pathlib import Path

import pandas as pd
from transformers import (DataCollatorForLanguageModeling, GPT2LMHeadModel,
                          GPT2Tokenizer, TextDataset, Trainer,
                          TrainingArguments)